## Tugas 5

In [1]:
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.4 MB/s eta 0:00:00


In [ ]:
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import pandas as pd
import numpy as np
import nltk
import re
import joblib

nltk.download('stopwords')
nltk.download('punkt')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/pramdf042/PPW/main/crawling_viva_fix.csv')
df

,Judul,Isi Berita,Link,Kategori
0,Terpopuler: Tentara Israel Frustasi Dihujat Wa...,Round Up – Perang Israel-Hamas di Jalur Gaza t...,https://www.viva.co.id/berita/nasional/1660151...,nasional
1,"Jadi Tersangka Pemerasan, Firli Bahuri Teranca...",Jakarta - Ketua Komisi Pemberantasan Korupsi (...,https://www.viva.co.id/berita/nasional/1660272...,nasional
2,"Firli Bahuri Ditetapkan Tersangka, Eks Pegawai...",Jakarta – Ketua wadah pegawai KPK Yudi Purnomo...,https://www.viva.co.id/berita/nasional/1660271...,nasional
3,Kondisi Terkini 4 Korban Luka Kecelakaan Minib...,Surabaya – Empat korban luka berat dalam insid...,https://www.viva.co.id/berita/nasional/1660237...,nasional
4,Polisi Segera Periksa Firli sebagai Tersangka ...,Jakarta- Polisi segera mengagendakan pemeriksa...,https://www.viva.co.id/berita/nasional/1660268...,nasional
...,...,...,...,...
67,Firli Bahuri Resmi Jadi Tersangka Kasus Pemera...,Jakarta – Polda Metro Jaya menetapkan Ketua Ko...,https://www.viva.co.id/berita/nasional/1660262...,kriminal
68,"Jadi Tersangka Pemerasan, Firli Bahuri Teranca...",Jakarta - Ketua Komisi Pemberantasan Korupsi (...,https://www.viva.co.id/berita/nasional/1660272...,kriminal
69,Kronologi Lengkap Kasus Mutilasi Mahasiswa UMY,Yogyakarta – Dua terdakwa kasus mutilasi terha...,https://www.viva.co.id/berita/kriminal/1660261...,kriminal
70,"Firli Bahuri Ditetapkan Tersangka, Eks Pegawai...",Jakarta – Ketua wadah pegawai KPK Yudi Purnomo...,https://www.viva.co.id/berita/nasional/1660271...,kriminal


In [ ]:
df.isnull().sum()

Judul         0
Isi Berita    0
Link          0
Kategori      0
dtype: int64

In [ ]:
df = df.dropna()
df.isnull().sum()

Judul         0
Isi Berita    0
Link          0
Kategori      0
dtype: int64

In [ ]:
df.shape

(72, 4)

In [ ]:
def cleaning(text):
  text = re.sub(r'[^\w\s.?!,]', '', text).strip().lower()
  return text

df['Cleaning'] = df['Isi Berita'].apply(cleaning)
df['Cleaning']

0     round up  perang israelhamas di jalur gaza ter...
1     jakarta  ketua komisi pemberantasan korupsi kp...
2     jakarta  ketua wadah pegawai kpk yudi purnomo ...
3     surabaya  empat korban luka berat dalam inside...
4     jakarta polisi segera mengagendakan pemeriksaa...
                            ...                        
67    jakarta  polda metro jaya menetapkan ketua kom...
68    jakarta  ketua komisi pemberantasan korupsi kp...
69    yogyakarta  dua terdakwa kasus mutilasi terhad...
70    jakarta  ketua wadah pegawai kpk yudi purnomo ...
71    pasuruan  seorang caleg muda pkb kabupaten pas...
Name: Cleaning, Length: 72, dtype: object

In [ ]:
def tokenizer(text):
  text = text.lower()
  return sent_tokenize(text)

df['Tokenizing'] = df['Cleaning'].apply(tokenizer)
df['Tokenizing']

0     [round up  perang israelhamas di jalur gaza te...
1     [jakarta  ketua komisi pemberantasan korupsi k...
2     [jakarta  ketua wadah pegawai kpk yudi purnomo...
3     [surabaya  empat korban luka berat dalam insid...
4     [jakarta polisi segera mengagendakan pemeriksa...
                            ...                        
67    [jakarta  polda metro jaya menetapkan ketua ko...
68    [jakarta  ketua komisi pemberantasan korupsi k...
69    [yogyakarta  dua terdakwa kasus mutilasi terha...
70    [jakarta  ketua wadah pegawai kpk yudi purnomo...
71    [pasuruan  seorang caleg muda pkb kabupaten pa...
Name: Tokenizing, Length: 72, dtype: object

In [ ]:
def graph_co_occurrence(x, threshold=0):
    vectorizer = CountVectorizer()
    tfidf_matrics = vectorizer.fit_transform(x)
    co_occurrence_matrix = tfidf_matrics.T.dot(tfidf_matrics).toarray()

    G = nx.Graph()

    # Menambahkan edge ke graf berdasarkan matriks co-occurrence
    for i in range(len(co_occurrence_matrix)):
      for j in range(i + 1, len(co_occurrence_matrix)):
        weight = co_occurrence_matrix[i, j]
        if weight > threshold:
          G.add_edge(vectorizer.get_feature_names_out()[i], vectorizer.get_feature_names_out()[j], weight=weight)
    return G

In [ ]:
def plot_graph(G, figsize=(35, 30), node_size=700, node_color='skyblue'):
  # Menggambar graf dengan canvas yang diperbesar
  pos = nx.spring_layout(G)  # Menentukan posisi simpul
  labels = nx.get_edge_attributes(G, 'weight')

  # Menentukan ukuran canvas
  plt.figure(figsize=(35, 30))

  # Menggambar graf dengan ukuran canvas yang diperbesar
  nx.draw(G, pos, with_labels=True, node_size=700, node_color='skyblue')
  nx.draw_networkx_edge_labels(G, pos, edge_labels=labels, font_color='red')
  plt.show()

In [ ]:
def text_rank_summarization(x, k=4, index=1, threshold=0.11, show_scores=False):
    G = graph_co_occurrence(x, threshold)

    # Menghitung nilai dari PageRank (TextRank)
    scores = nx.pagerank(G)

    # Menyusun Kalimat berdasarkan nilai TextRank Scores tertinggi
    ranked_sentences = sorted(((sum(scores.get(word, 0) for word in kalimat.split()), kalimat) for kalimat in x), reverse=True)

    if show_scores:
      print(f"TextRank Scores Dokumen Ke - {index} : {scores}")
      # Menampilkan hasil peringkatan kalimat
      for skor, kalimat in ranked_sentences:
          print(f"Skor: {skor}, Kalimat: {kalimat}")

    summary_sentences = [s for _, s in ranked_sentences[:k]]

    return (' '.join(summary_sentences), G)

In [ ]:
summary, G = text_rank_summarization(df["Tokenizing"].iloc[:1][0], k = 3, show_scores=True, threshold=0)

TextRank Scores Dokumen Ke - 1 : {'100': 0.0034175745128025414, '12': 0.0034175745128025414, 'berbekal': 0.0034175745128025414, 'berusia': 0.0034175745128025414, 'bocah': 0.005969745195725754, 'dekatnya': 0.0034175745128025414, 'di': 0.03987438061072995, 'dua': 0.005969745195725753, 'hanya': 0.00921585935136173, 'itu': 0.013059177961291938, 'jakarta': 0.005969745195725753, 'kerabat': 0.0034175745128025414, 'kilometer': 0.0034175745128025414, 'masih': 0.01034244745513185, 'menempuh': 0.0034175745128025414, 'menemui': 0.0034175745128025414, 'perjalanan': 0.0034175745128025414, 'ratusan': 0.0034175745128025414, 'ribu': 0.0034175745128025414, 'rp': 0.0034175745128025414, 'tahun': 0.0034175745128025414, 'uang': 0.0034175745128025414, 'untuk': 0.0077892984756364235, 'yang': 0.009395732034581628, '128': 0.003303498433075292, '14': 0.003303498433075292, '600': 0.003303498433075292, 'adalah': 0.003303498433075292, 'agresi': 0.003303498433075292, 'anakanak': 0.003303498433075292, 'demikian': 0.0

In [ ]:
summary

'berita seputar israel dan isu terkini di tanah air masih menjadi tajuk terpopuler di laman news viva, rabu, 23 november 2023. simak beritaberita pilihan itu dalam round up berikut ini\nmasyarakat indonesia ini menggunakan jarijari mereka untuk menyerang akun milik pasukan zionis. tak hanya aksi protes dan boikot produk israel menggema di berbagai belahan dunia, kecaman atas aksi tak berperikemanusiaan militer israel tak hanya di darat, juga di jagat maya. selengkapnya di tautan ini\nkoordinator masyarakat anti korupsi maki boyamin saiman menyebutkan kalau ketua kpk firli bahuri berbohong ketika dirinya menjelaskan mobilnya hilang saat rampung menjalani pemeriksaan di bareskrim polri.'

In [ ]:
plot_graph(G)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
df["Summary"] = df.apply(lambda row : text_rank_summarization(row['Tokenizing'], index=row.name+1, k=7, show_scores=True)[0], axis=1)

TextRank Scores Dokumen Ke - 1 : {'100': 0.0034175745128025414, '12': 0.0034175745128025414, 'berbekal': 0.0034175745128025414, 'berusia': 0.0034175745128025414, 'bocah': 0.005969745195725754, 'dekatnya': 0.0034175745128025414, 'di': 0.03987438061072995, 'dua': 0.005969745195725753, 'hanya': 0.00921585935136173, 'itu': 0.013059177961291938, 'jakarta': 0.005969745195725753, 'kerabat': 0.0034175745128025414, 'kilometer': 0.0034175745128025414, 'masih': 0.01034244745513185, 'menempuh': 0.0034175745128025414, 'menemui': 0.0034175745128025414, 'perjalanan': 0.0034175745128025414, 'ratusan': 0.0034175745128025414, 'ribu': 0.0034175745128025414, 'rp': 0.0034175745128025414, 'tahun': 0.0034175745128025414, 'uang': 0.0034175745128025414, 'untuk': 0.0077892984756364235, 'yang': 0.009395732034581628, '128': 0.003303498433075292, '14': 0.003303498433075292, '600': 0.003303498433075292, 'adalah': 0.003303498433075292, 'agresi': 0.003303498433075292, 'anakanak': 0.003303498433075292, 'demikian': 0.0

In [ ]:
new_df = df[["Summary", "Kategori"]]
new_df

,Summary,Kategori
0,berita seputar israel dan isu terkini di tanah...,nasional
1,jakarta ketua komisi pemberantasan korupsi kp...,nasional
2,"sebelumnya diberitakan, polda metro jaya menet...",nasional
3,"yang dua kondisinya tidak memungkinkan, karena...",nasional
4,"sebelumnya diberitakan, polda metro jaya menet...",nasional
...,...,...
67,jakarta polda metro jaya menetapkan ketua kom...,kriminal
68,jakarta ketua komisi pemberantasan korupsi kp...,kriminal
69,yogyakarta dua terdakwa kasus mutilasi terhad...,kriminal
70,"sebelumnya diberitakan, polda metro jaya menet...",kriminal


In [ ]:
df["Summary"][1]

'jakarta\xa0 ketua komisi pemberantasan korupsi kpk, firli bahuri terancam pidana penjara seumur hidup buntut ditetapkan jadi tersangka kasus dugaan pemerasan atau penerimaan gratifikasi dalam penanganan korupsi kementerian pertanian kementan 2021.\nfirli bakal dikenakan pasal 12e atau pasal 12b atau pasal 11 uu 311999 yang telah diubah dengan uu 202001 tentang perubahan atas uu 311999 tentang pemberantasan tindak pidana korupsi juncto pasal 65 kuhp. untuk diketahui, polda metro jaya menaikkan status kasus dugaan pemerasan yang dilakukan pimpinan komisi pemberantasan korupsi kpk dalam penanganan kasus korupsi di kementerian pertanian kementan tahun 2021 ke penyidikan. dalam rentetan pasal berlapis itu, firli terancam hukuman paling berat penjara seumur hidup sebagaimana pasal 12b ayat 1 terkait penerimaan gratifikasi yang dilakukan bagi pegawai negeri atau penyelenggara negara. dan pidana denda paling sedikit rp200 juta dan paling banyak rp1 miliar, kata dia kepada wartawan, kamis 23 n

In [ ]:
vectorizer_summary = TfidfVectorizer()
X_summary = vectorizer_summary.fit_transform(df["Summary"]).toarray()
y = df["Kategori"]

In [ ]:
X_train_summary, X_test_summary, y_train_summary, y_test_summary = train_test_split(X_summary, y, test_size=0.3, random_state=42)

In [ ]:
# Inisialisasi model Naive Bayes Gaussian
gnb_summary = GaussianNB()

# Melatih model menggunakan data latih
gnb_summary.fit(X_train_summary, y_train_summary)

# Membuat prediksi pada data uji
y_pred_gnb_summary = gnb_summary.predict(X_test_summary)

In [ ]:
# Mengukur kinerja model dengan menghitung akurasi atau metrik lain yang relevan
accuracy = accuracy_score(y_test_summary, y_pred_gnb_summary)
print(f'Akurasi: {accuracy}')

Akurasi: 0.5454545454545454


In [ ]:
new_data = ["Sebuah tim peneliti, yang dipimpin oleh mahasiswa doktoral Institute of Agricultural Engineering, University of Bonn, Jerman Sonja Hillemacher dan Dr. Inga Tiemann, telah melakukan penelitian ilmiah selama bertahun-tahun terhadap perilaku ayam di depan cermin. Ide bereksperimen dengan ayam datang dari para ilmuwan tersebut bersama guru besar Biopsikologi dari Departemen Biopsikologi di Universitas Ruhr, Onur Güntürkün. Filosofi penelitian ini bermula dari sebuah pertanyaan terkait 'Apakah hewan dapat mengenali dirinya sendiri?,' kata Sonja Hillemacher. Penelitian ini pun bertujuan untuk menjawab pertanyaan apakah ayam mampu mengenali diri mereka sendiri ketika berinteraksi dengan cermin. Metode Pendekatan Perilaku Hewan Penelitian ini mengadopsi metode yang telah umum digunakan dalam uji cermin, yang dikenal sebagai 'mark test'. Dalam tes ini, tanda berwarna ditempatkan di tubuh hewan yang hanya dapat dikenali oleh individu tersebut ketika ia melihat dirinya sendiri di cermin. Apabila hewan tersebut mulai memeriksa atau meraba area tubuhnya yang ditandai di depan cermin, itu menunjukkan bahwa hewan telah mengenali wujudnya sebagai dirinya sendiri. Namun, tes ini tidak selalu berhasil, di mana sebagian hewan yang diyakini sadar diri, tidak memperhatikan cermin. Mungkin, karena mereka merasa tidak nyaman dalam lingkungan percobaan 'buatan'? Penelitian ini mencoba mendekati eksperimen ini dengan mempertimbangkan perilaku alami ayam yang relevan secara ekologis. 'Beberapa ayam, terutama ayam jantan, memperingatkan sesamanya melalui panggilan khusus ketika ada predator -- seperti burung pemangsa atau rubah - - muncul,' kata Güntürkün. Sebaliknya jika ayam jago berhadapan dengan predatornya sendirian, biasanya mereka akan berdiam diri agar tidak menarik perhatian predator itu sendiri dan menjadi korban. 'Seruan peringatan adalah perilaku sempurna untuk diintegrasikan ke dalam tes kesadaran diri yang lebih relevan secara ekologis,' tambah biopsikolog tersebut. Uji Coba dengan Melepas Burung Pemangsa Pertama, penelitian ini memeriksa apakah ayam jantan benar-benar mengeluarkan panggilan alarm saat berhadapan dengan spesies dan tetap diam saat sendirian. Ilmuwan membangun tempat uji coba khusus yang memisahkan dua area di mana ayam jantan dapat melihat satu sama lain. Setelah itu, seekor burung pemangsa pun dilepaskan di langit-langit sebuah area. Dengan melibatkan 58 ekor ayam jantan, setiap ayamnya mengalami proses pengulangan penelitian sebanyak tiga kali untuk memvalidasi hasil statistik. Hasilnya pun menunjukkan bahwa ayam jantan mengeluarkan 77 panggilan alarm saat berhadapan dengan ayam jantan lain, tetapi hanya 17 panggilan ketika ayam jantan itu sendiri. Dengan begitu, ini menunjukkan bahwa sebagian besar ayam jantan sangat waspada saat menghadapi sesamanya ketika predator terlihat. Uji Coba dengan Cermin Langkah selanjutnya adalah menempatkan cermin di antara dua kompartemen, bukan di dalam kisi-kisi. Dalam proses penelitian ini, para peneliti menjalankannya tiga kali dengan masing-masing hewan. Hasilnya, hanya 25 panggilan alarm yang dikeluarkan selama 174 percobaan. Hasil ini mendukung gagasan bahwa ayam jantan mungkin mengenali dirinya sendiri dalam bayangan cerminnya. Namun, masih diperlukan penelitian lebih lanjut untuk memahami apakah hewan-hewan tersebut melihat bayangan diri mereka sebagai entitas yang sama atau sebagai binatang asing yang meniru perilaku mereka sendiri. Penelitian ini menunjukkan bahwa uji tanda cermin yang mengintegrasikan perilaku hewan ke dalam konteks yang lebih relevan secara ekologis memberikan hasil yang lebih dapat diandalkan."]
new_data_summary, G = text_rank_summarization(tokenizer(new_data[0]), k=5)
new_data_summary

'penelitian ini menunjukkan bahwa uji tanda cermin yang mengintegrasikan perilaku hewan ke dalam konteks yang lebih relevan secara ekologis memberikan hasil yang lebih dapat diandalkan. hasilnya pun menunjukkan bahwa ayam jantan mengeluarkan 77 panggilan alarm saat berhadapan dengan ayam jantan lain, tetapi hanya 17 panggilan ketika ayam jantan itu sendiri. uji coba dengan melepas burung pemangsa pertama, penelitian ini memeriksa apakah ayam jantan benar-benar mengeluarkan panggilan alarm saat berhadapan dengan spesies dan tetap diam saat sendirian. namun, masih diperlukan penelitian lebih lanjut untuk memahami apakah hewan-hewan tersebut melihat bayangan diri mereka sebagai entitas yang sama atau sebagai binatang asing yang meniru perilaku mereka sendiri. sebuah tim peneliti, yang dipimpin oleh mahasiswa doktoral institute of agricultural engineering, university of bonn, jerman sonja hillemacher dan dr. inga tiemann, telah melakukan penelitian ilmiah selama bertahun-tahun terhadap per

In [ ]:
tfidf_Xnew_summary = vectorizer_summary.transform([new_data_summary]).toarray()
tfidf_Xnew_summary

In [ ]:
pred_gnb_summary = gnb_summary.predict(tfidf_Xnew_summary)
pred_gnb_summary[0]

In [ ]:
joblib.dump(gnb_summary, 'modelNB.pkl')